In [31]:
import json
import os
import numpy as np
import math
from typing import Dict, List, Tuple
import copy

from datetime import timedelta
from config import *
from utils import *
from main import *
from GtConflictFinder import GtConflictFinder

import plotly.graph_objects as go
from plotly.subplots import make_subplots
test_projects = [
    "HomeA/2016",
    "HomeB/2016",
    "HomeC/2016",
    "HomeD/2016",
    "HomeF/2016",
    "HomeG/2016",
]
ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 20,
            },
            # "humidity#NUM" : {
            #     "range" : (0., 1.0),
            #     "interval" : 0.1,
            # },
            WEEKDAY_CTX: {
                "range": (0, 6.1),
                "interval": 1,
            },
        })
capacity = {
    "Range" : 2,
    "Microwave": 1,
    "LivingLights": 0,
    "HomeOffice": 1,
    "WashingMachine": 1,
}


In [32]:
habit_groups = {}
for p in test_projects:
    habit_groups[p], _ = test_umass(test_project=p, ctx_info=ctx_info)

DEBUG:root:The number of device events from processed file: {'Range': 625, 'Microwave': 1591, 'LivingLights': 1609, 'HomeOffice': 1481}
DEBUG:root:The number of context events from processed file: {'apparentTemperature#NUM': 8738, 'temperature#NUM': 8735, 'humidity#NUM': 7642, 'summary#CAT': 2669}
DEBUG:root:The number of device events from processed file: {'LivingLights': 1923, 'Microwave': 863}
DEBUG:root:The number of context events from processed file: {'apparentTemperature#NUM': 8734, 'temperature#NUM': 8737, 'humidity#NUM': 7657, 'summary#CAT': 1799}
DEBUG:root:The number of device events from processed file: {'HomeOffice': 968, 'Microwave': 2059, 'LivingLights': 1759}
DEBUG:root:The number of context events from processed file: {'apparentTemperature#NUM': 8736, 'temperature#NUM': 8735, 'humidity#NUM': 7618, 'summary#CAT': 1863}
DEBUG:root:The number of device events from processed file: {'Microwave': 1423, 'LivingLights': 1159, 'WashingMachine': 313, 'Range': 1163}
DEBUG:root:Th

In [33]:
#Exps:
#   1. Contrived environment with injected conflicts:
#   2. Real dataset with groundtruth conflicts:
#   3. observations about the algorithm: with different threshold, different capacity for the devices, different number of users.

In [34]:
for home, groups in habit_groups.items():
    print("The habit groups found in {}".format(home))
    for d, d_groups in groups.items():
        print(d + " " + str(len(d_groups)))


The habit groups found in HomeA/2016
Range 10
Microwave 8
LivingLights 72
HomeOffice 22
The habit groups found in HomeB/2016
LivingLights 37
Microwave 3
The habit groups found in HomeC/2016
HomeOffice 35
Microwave 10
LivingLights 54
The habit groups found in HomeD/2016
Microwave 6
LivingLights 25
WashingMachine 8
Range 15
The habit groups found in HomeF/2016
Microwave 15
WashingMachine 11
The habit groups found in HomeG/2016
LivingLights 11
HomeOffice 32
Range 7
Microwave 9


In [35]:
 
from rtree import index
from itertools import combinations

def get_bound(box):
    mid = len(box) // 2
    return box[:mid], box[mid:]

def compute_intersection_area(box_i, box_j):
    # This method assume the input boxes does intersect
    mins_i, maxs_i = get_bound(box_i)
    mins_j, maxs_j = get_bound(box_j)
    mins = [max(i,j) for i,j in zip(mins_i, mins_j)]
    maxs = [min(i,j) for i,j in zip(maxs_i, maxs_j)]
    return mins+maxs

def does_contain(box_i, box_j):
    mins_i, maxs_i = get_bound(box_i)
    mins_j, maxs_j = get_bound(box_j)
    for i in range(len(mins_i)):
        if mins_i[i] < mins_j[i]  or maxs_i[i] > maxs_j[i]:
            return False
    return True

def device_state_conflict(dis_i, dis_j):
    total_j = sum(dis_j)
    prob = 0
    for idx, p in enumerate(dis_i):
        prob += p * (total_j -  dis_j[idx])
    return prob

def device_capacity_conflict(dis, capacity):
    probs = [sum(x) for x in dis]
    ids = range(len(dis))
    prob = 1
    for i in range(0, capacity+1):
        cbs = list(combinations(ids, i))
        for c in cbs:
            pp = 1
            for i, p in enumerate(probs):
                pp *= p if i in c else (1-p)
            prob -= pp
    return prob

users = list(habit_groups.keys())
p = index.Property()
p.dimension = len(ctx_info.get_all_ctx_ordered())
print(p.dimension)
final_conflicts = {x:{
    "DiffState": [],
    "Capacity": [],
} for x in capacity}
capacity_conflict_tree = {x:index.Index(properties=p) for x in capacity}
capacity_conflict_id = {x:0 for x in capacity}
min_conflict_prob = 0.01
for user_i in range(len(users)):
# for user_i in [0]:
    for device, groups in habit_groups[users[user_i]].items():
        r_tree = index.Index(properties=p)
        for i in range(len(groups)):
            bound = groups[i]["box"][0] + groups[i]["box"][1] 
            r_tree.insert(id=i, coordinates=bound, obj={(user_i, i)})
        tree_id = len(groups)
        for user_j in range(user_i+1, len(users)):
            if device not in habit_groups[users[user_j]]:
                # This user does not have this device
                continue    
            j_groups = habit_groups[users[user_j]][device]
            for j, g in enumerate(j_groups):
                bbox = g["box"][0] + g["box"][1]
                intersects = list(r_tree.intersection(bbox, objects=True))
                for intersect in intersects:
                    intersect_box = compute_intersection_area(intersect.bbox, bbox)
                    dis = intersect.object.union({(user_j, j)})
                    r_tree.insert(id=tree_id, coordinates=intersect_box, obj=dis)
                    tree_id += 1
        # Use the original box to intersect all
        for i in range(len(groups)):
            bound = groups[i]["box"][0] + groups[i]["box"][1] 
            intersects = list(r_tree.intersection(bound, objects=True))
            # intersects = sorted(intersects, key = lambda x: len(x.object))

            # First we calculate conflicts between two users
            for ints in intersects:
                if len(ints.object) == 2:
                    # compute conflict between two users
                    dis = [
                        habit_groups[users[x[0]]][device][x[1]]["dis"]
                        for x in ints.object
                    ]
                    prob = device_state_conflict(dis[0], dis[1])
                    if prob > min_conflict_prob:
                        final_conflicts[device]["DiffState"].append({
                            "users": ints.object,
                            "box": ints.bbox,
                            "prob": prob,
                        })
                if len(ints.object) > capacity[device] and capacity[device] != 0:
                    # Put capacity conflicts into a R-tree for further analysis
                    cap_box = ints.bbox
                    cap_ints = list(capacity_conflict_tree[device].intersection(cap_box, objects=True))
                    box_to_insert = ints.bbox
                    obj_to_insert = ints.object
                    for inter_box in cap_ints:
                        if does_contain(cap_box, inter_box.bbox):
                            capacity_conflict_tree[device].delete(inter_box.id, inter_box.bbox)
                            dis_union = inter_box.object.union(ints.object)
                            box_to_insert = inter_box.bbox
                            obj_to_insert = dis_union
                        if does_contain(inter_box.bbox, cap_box):
                            capacity_conflict_tree[device].delete(inter_box.id, inter_box.bbox)
                            dis_union = inter_box.object.union(ints.object)
                            box_to_insert = cap_box
                            obj_to_insert = dis_union

                    capacity_conflict_tree[device].insert(
                        id=capacity_conflict_id[device],
                        coordinates=box_to_insert, 
                        obj = obj_to_insert,
                    )
                    capacity_conflict_id[device] += 1
                
print(capacity_conflict_id)
max_box = [0]*len(ctx_info.get_ctx_space_shape()) + [
    x - 1
    for x in ctx_info.get_ctx_space_shape()
]
for d, tree in capacity_conflict_tree.items():
    conflicts = list(tree.intersection(max_box, objects=True))
    for c in conflicts:
        dis = [ 
            habit_groups[users[x[0]]][d][x[1]]["dis"]
            for x in c.object
        ]
        prob = device_capacity_conflict(dis, capacity[d])
        if prob > min_conflict_prob:
            final_conflicts[d]["Capacity"].append({
                "users": c.object,
                "box": c.bbox,
                "prob": prob,
            })
print({x:len(final_conflicts[x]["Capacity"]) for x in final_conflicts})

2
{'Range': 49, 'Microwave': 2004, 'LivingLights': 0, 'HomeOffice': 315, 'WashingMachine': 23}
{'Range': 0, 'Microwave': 10, 'LivingLights': 0, 'HomeOffice': 34, 'WashingMachine': 3}


In [36]:
import plotly.express as px
probs = [(x["box"],x["prob"]) for x in final_conflicts["Microwave"]["Capacity"]]
print(probs)
# fig = px.histogram(probs)
# fig.show()

[([60.0, 6.0, 63.0, 6.0], 0.01667065286318719), ([40.0, 0.0, 52.0, 3.0], 0.16301348761816012), ([37.0, 0.0, 43.0, 0.0], 0.01067120761841214), ([18.0, 4.0, 27.0, 6.0], 0.048572183804670065), ([40.0, 4.0, 52.0, 6.0], 0.284906712873913), ([28.0, 0.0, 39.0, 3.0], 0.01971688643413323), ([53.0, 0.0, 59.0, 3.0], 0.04892256164828413), ([53.0, 0.0, 53.0, 4.0], 0.011699922341680685), ([53.0, 4.0, 55.0, 6.0], 0.08640197893077751), ([28.0, 4.0, 39.0, 6.0], 0.08983047495783551)]


In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

draw = {
    "HomeA/2016": ["Range"],
    "HomeG/2016": ["Range"],
}
fig = make_subplots(rows=sum([len(draw[x]) for x in draw]), cols=1)
row_id = 1
for home in draw:
    for d in draw[home]:
        r_tree = index.Index(properties=p)
        for i,x in enumerate(habit_groups[home][d]):
            bound = x["box"][0] + x["box"][1] 
            r_tree.insert(id=i, coordinates=bound, obj=sum(x["dis"]))
        on_rate_group = np.full(ctx_info.get_ctx_space_shape(), -0.1)

        for i in range(ctx_info.get_ctx_space_shape()[0]):
            for j in range(ctx_info.get_ctx_space_shape()[1]):
                result = list(r_tree.intersection([i,j,i,j], objects=True))

                on_rate_group[i,j] = result[0].object
        fig.append_trace(
            go.Heatmap(z=np.transpose(on_rate_group)),
            row_id, 1
        )
        row_id += 1
fig.show()

RTreeError: Coordinates must be in the form (minx, miny, maxx, maxy) or (x, y) for 2D indexes

In [37]:

a = list(idx.intersection(box,  objects=True))
print(a[0].id)

AttributeError: 'int' object has no attribute 'intersection'

In [46]:
import copy
# Make ground truth:
ratio = 0.3
device_events = {}
for p in test_projects:
    ctx_evts, device_evts = load_processed(p)
    device_events[p] = device_evts

gtconflict_cfg = {
    "context_info": ctx_info,
    "capacity": capacity
}

conflict_finder = GtConflictFinder(gtconflict_cfg)
conflicts = conflict_finder.get_Gt_conflict(ctx_evts, device_events)
print(len(conflicts))

868


In [47]:
import plotly.express as px

TEST_RATIO = 0.3
conflict_time = [x['cur_time'] for x in conflicts]
end_time = max(conflict_time)
start_time = min(conflict_time)
print(start_time)
print(end_time)
total_time_range = end_time - start_time
test_start = end_time - total_time_range * TEST_RATIO
print(test_start)

conflict_device = {
    d:[]
    for d in capacity
}
for c in conflicts:
    if c["cur_time"] > test_start:
        conflict_device[c["device"]].append(c)
print({d:len(c) for d,c in conflict_device.items()})


2016-01-01 15:07:00
2016-12-29 16:48:00
2016-09-11 18:41:42
{'Range': 0, 'Microwave': 194, 'LivingLights': 0, 'HomeOffice': 91, 'WashingMachine': 4}


In [48]:
def does_contain_point(box, point):
    mins_i, maxs_i = get_bound(box)
    for i in range(len(mins_i)):
        if point[i] < mins_i[i] or point[i] > maxs_i[i]:
            return False
    return True

missed_gt_conflicts = {d:[] for d in conflict_device}
hit_gt_conflicts = {d:{} for d in conflict_device}
for d in final_conflicts:
    hit_gt_conflicts[d] = {i:0 for i in range(len(final_conflicts[d]["Capacity"]))}
for d, conflicts in conflict_device.items():
    for c in conflicts:
        coor = ctx_info.get_coor_by_ctx(c["ctx"])
        flag = False
        for idx,c_predict in enumerate(final_conflicts[d]["Capacity"]):
            if does_contain_point(c_predict["box"], coor):
                flag = True
                hit_gt_conflicts[d][idx] += 1
        if not flag:
            missed_gt_conflicts[d].append(c)
print({d:len(missed_gt_conflicts[d]) for d in missed_gt_conflicts})
hit_count = {d:[0,0] for d in hit_gt_conflicts}
for d,hit_c in hit_gt_conflicts.items():
    for i,h in hit_c.items():
        if h == 0:
            hit_count[d][0] += 1
        else:
            hit_count[d][1] += 1
print(hit_count)

{'Range': 0, 'Microwave': 36, 'LivingLights': 0, 'HomeOffice': 16, 'WashingMachine': 1}
{'Range': [0, 0], 'Microwave': [0, 10], 'LivingLights': [0, 0], 'HomeOffice': [7, 27], 'WashingMachine': [0, 3]}
